In [1]:
# ---------------- import necessary libraries ---------------------------
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from numpy import atleast_2d as twod
import mltools as ml
np.random.seed(0)


In [27]:
# Data Loading
data = np.genfromtxt('data/citrus.csv', delimiter=",", skip_header = 1,
                     dtype=["|S5"] + ["float" for n in range(5)])

np.random.shuffle(data)

data = np.array(data.tolist())
X = np.asarray([[float(y) for y in x] for x in data[:,1:]], dtype = float)
Y = np.asarray([str(y,'utf-8') for y in data[:,0]], dtype = str)

for i in X:
    for index, x in enumerate(i):
        if(x == None):
            x = np.mean(X[:,index]) 



data_size = X.shape[0]
first_half = round(int(data_size / 2))
third = first_half + round((first_half / 2))

Xtr = X[0:third, :]
Ytr = Y[0:third]
Xte = X[third:-1, :]

#convert target values to binary values
for y in range(Ytr.shape[0]):
    if(str(Ytr[y]) == 'orang'):
        Ytr[y] = int(0)
    elif(str(Ytr[y]) == 'grape'):
        Ytr[y] = int(1)
    else:
        Ytr[y] = int(1)

# Split into training and validation data
Xtr,Xva,Ytr,Yva = ml.splitData(Xtr,Ytr,0.75) # split data set 75/25

def toKaggle(filename,YeHat):
    fh=open(filename,'w')# open file for upload
    fh.write('ID,Predicted\n')# output header line
    for i,yi in enumerate(YeHat.ravel()):
        fh.write('{},{}\n'.format(i,yi))# output each prediction
    fh.close()

print("Done Loading Data")

Done Loading Data


In [165]:
# Simple Neural Network Classifier

sig_0 = lambda z: twod(1 / (1 + np.exp(-z)))
dSig_0 = lambda z: twod(np.multiply(sig(z), (1 - sig(z))))
sig = lambda z: twod(np.tanh(z))
dSig = lambda z: twod(1 - np.power(np.tanh(z), 2))

nn = ml.nnet.nnetClassify()
nn.init_weights([Xtr.shape[1], 200, len(np.unique(Ytr))], 'random', Xtr, Ytr)
nn.setActivation('logistic')
nn.train(Xtr,Ytr,stopTol=1e-5,stepsize=0.6,stopIter= 5000)

PvaNn=nn.predictSoft(Xva)
PteNn=nn.predictSoft(Xte)

toKaggle('PvaNn.csv',PvaNn[:,1])
toKaggle('PteNn.csv',PteNn[:,1])

print("AUC of Neural Network is: {}".format(nn.auc(Xva,Yva)))


it 1 : Jsur = 0.19501343903711738, J01 = 0.11697777777777778
it 2 : Jsur = 0.1875458982978953, J01 = 0.11235555555555556
it 4 : Jsur = 0.17716184931514847, J01 = 0.10488888888888889
it 8 : Jsur = 0.17253149738452841, J01 = 0.104
it 16 : Jsur = 0.16969255189695273, J01 = 0.10293333333333334
AUC of Neural Network is: 0.9244548924808281


In [167]:

AUCva = []

esemble = [0] * 30 
current = 0

for i in range(30):
    Xi,Yi = ml.bootstrapData(Xtr,Ytr, n_boot = 3000)
    
    learner = ml.dtree.treeClassify(Xi, Yi, minLeaf = 30, maxDepth = 19,
    nFeatures = 70, minParent = 220)
    
    esemble[current] = learner
    AUCva.append(learner.auc(Xva, Yva))
    current = current + 1

meanAUC = np.mean(AUCva)
print("AUC of Random Forest is: {}".format(meanAUC))

test_predictions = np.zeros((Xte.shape[0], 2))
va_predictions = np.zeros((Xva.shape[0], 2))
for i in range(30):
    test_predictions += esemble[i].predictSoft(Xte)
    va_predictions += esemble[i].predictSoft(Xva)

test_predictions /= len(keep)
va_predictions /= len(keep)

toKaggle('Pva1.csv',va_predictions[:,1])
toKaggle('Pte1.csv',test_predictions[:,1])


AUC of Random Forest is: 0.9659784013214451


In [168]:

Pva1=np.genfromtxt('Pva1.csv',delimiter=',',skip_header=1)[:,1:2]
Pte1=np.genfromtxt('Pte1.csv',delimiter=',',skip_header=1)[:,1:2]
PvaNn=np.genfromtxt('PvaNn.csv',delimiter=',',skip_header=1)[:,1:2]
PteNn=np.genfromtxt('PteNn.csv',delimiter=',',skip_header=1)[:,1:2]

Sv=np.hstack((Pva1,PvaNn))
stack= ml.linearC.linearClassify(Sv,Yva,reg=1e-6)
print("** Stacked AUC: ",stack.auc(Sv,Yva))

Se=np.hstack((Pte1,PteNn))
PeS=stack.predictSoft(Se)
toKaggle('Ex_Stack.csv',PeS[:,1])

** Stacked AUC:  0.9858758026664118
